# Dependences

In [16]:
include("../../src/struct_data.jl")

fusion_agg (generic function with 1 method)

# Input Data

# Model Parameter

In [17]:
@make_struct_func Cubic
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

  0.000002 seconds (3 allocations: 112 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"


# Aggregate

## Fusion Aggs

### First

In [18]:
AGG = Nothing
@time AGG = Aggregate(
    "HEK", 
    InteractionPar(
        Cubic(1.0,2.0,3.0), 
        ContractilePar(0.01)
    ),
    Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu, 
    Model
)
println("===================================== One Aggregate =====================================")
show_aggregates(AGG)

  0.150615 seconds (326.62 k allocations: 18.137 MiB)
===================================== One Aggregate =====================================
------------------ Agg Properties -------------------
Name    = HEK
Radius  = 15.270000457763672
InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.01
--------------------- Position ----------------------


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -13.88
  0.5  -4.62  -13.88
  2.5  -4.62  -13.88
 -4.5  -2.89  -13.88
 -2.5  -2.89  -13.88
 -0.5  -2.89  -13.88
  1.5  -2.89  -13.88
  3.5  -2.89  -13.88
 -5.5  -1.15  -13.88
 -3.5  -1.15  -13.88
 -1.5  -1.15  -13.88
  0.5  -1.15  -13.88
  2.5  -1.15  -13.88
  ⋮           
 -1.5   1.15   13.88
  0.5   1.15   13.88
  2.5   1.15   13.88
  4.5   1.15   13.88
 -4.5   2.89   13.88
 -2.5   2.89   13.88
 -0.5   2.89   13.88
  1.5   2.89   13.88
  3.5   2.89   13.88
 -1.5   4.62   13.88
  0.5   4.62   13.88
  2.5   4.62   13.88

--------------------- Outline -----------------------


1×2504 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204


In [19]:
println("=================================== Fusion Aggregate ====================================")
FusionAGG = Nothing
@time FusionAGG = fusion_agg(AGG)
show_aggregates(FusionAGG)

TEXT = hcat(FusionAGG.Outline,Matrix(FusionAGG.Position))
open("tetero.xyz", "w") do f
    write(f, "$(size(FusionAGG.Position, 1))\n")
    write(f, "t=$(0)\n")
    writedlm(f,TEXT, ' ')
end

=================================== Fusion Aggregate ====================================
  0.167029 seconds (192.95 k allocations: 11.151 MiB, 93.59% compilation time)
---------------- Aggs Type List ---------------------


1×4 Matrix{Any}:
 1  …  InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01))

------------------- Aggs Index ----------------------
Index of List of Aggregates


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Index of Number of Aggregates


...

--------------------- Position ----------------------


5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -16.77  -4.62  -13.88
 -14.77  -4.62  -13.88
 -12.77  -4.62  -13.88
 -19.77  -2.89  -13.88
 -17.77  -2.89  -13.88
 -15.77  -2.89  -13.88
 -13.77  -2.89  -13.88
 -11.77  -2.89  -13.88
 -20.77  -1.15  -13.88
 -18.77  -1.15  -13.88
 -16.77  -1.15  -13.88
 -14.77  -1.15  -13.88
 -12.77  -1.15  -13.88
   ⋮            
  13.77   1.15   13.88
  15.77   1.15   13.88
  17.77   1.15   13.88
  19.77   1.15   13.88
  10.77   2.89   13.88
  12.77   2.89   13.88
  14.77   2.89   13.88
  16.77   2.89   13.88
  18.77   2.89   13.88
  13.77   4.62   13.88
  15.77   4.62   13.88
  17.77   4.62   13.88

--------------------- Outline -----------------------


1×5008 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204


# Simulation Struct

In [20]:
Simulation = Nothing
@time Simulation = SimulationSet(FusionAGG, Model)
show_simulation_set(Simulation)

  0.313857 seconds (53.11 k allocations: 294.650 MiB, 12.01% gc time, 17.96% compilation time)
============== Type of Matrix =============
CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}
========== Neighbors Matrix Size ==========
idx      = (5008, 5008)
idx_red  = (21, 5008)
idx_sum  = (1, 5008)
idx_cont = (50, 5008)
============ Forces Matrix Size ===========
dX       = (5008, 3)
F       = (5008, 3)
